In [ ]:
# %load "firstcell.py"
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot

from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

In [ ]:
import pathlib
import awkward
import json
import bitstruct
import crcmod
import scipy.stats
from tqdm.notebook import trange, tqdm

from ECOND_SEU import run_times, fluences, hexa42_files, hexprint, items, packet_parser, capture_batch, incident

In [ ]:
# This is the same as the `incident` class, except for how we provide the inputs
class incident2:
    def __init__(self, ASIC, EMU, cap_rows, packet, prebeam):
        if packet == 255:
            packet_rows = cap_rows
        else:
            packet_rows = (prebeam.packet_num == packet).nonzero()[0]
        
        first_row = min(cap_rows[0], packet_rows[0])
        last_row = max(cap_rows[-1]+1, packet_rows[-1] + 1)
        self.N_row = last_row - first_row
        
        self.prebeam_packet = prebeam.prebeam_data[first_row : last_row]
        
        self.ASIC_packet = zeros_like(self.prebeam_packet)
        self.EMU_packet  = zeros_like(self.prebeam_packet)
        self.ASIC_packet[cap_rows - first_row] = ASIC
        self.EMU_packet [cap_rows - first_row] = EMU
        
        self.xor = self.ASIC_packet ^ self.EMU_packet
        
        self.cap = zeros_like(self.prebeam_packet, bool)
        self.cap[cap_rows - first_row] = True
        
        self.gap = zeros_like(self.prebeam_packet, bool)
        gap_indices = cap_rows - first_row + 4
        gap_indices = gap_indices[gap_indices < (last_row - first_row)]
        self.gap[gap_indices] = True
        
        self.known = self.cap | ~self.gap
        self.mismatch = self.ASIC_packet != self.EMU_packet
        code = prebeam.code_data[first_row : last_row]
        subpack = prebeam.subpack[first_row : last_row]
        subcode = prebeam.subcode[first_row : last_row]
        subp_any = prebeam.subp_any[first_row : last_row]
        
        tri_logic = 1 * ~self.known + 2 * self.mismatch
        
        idle = tri_logic[code == 0].max(initial=0)
        
        packet_header_0 = tri_logic[code == 1].max(initial=0)
        Payload_Length = any((self.xor[code == 1] & 0x007fc000) != 0) * packet_header_0
        Header_Hamming = any((self.xor[code == 1] & 0x0000003f) != 0) * packet_header_0
        
        packet_header_1 = tri_logic[code == 2].max(initial=0)
        L1A_orbit    = any((self.xor[code == 2] & 0x000ff800) != 0) * packet_header_1
        Header_CRC   = any((self.xor[code == 2] & 0x000000ff) != 0) * packet_header_1
        
        Event_Header = max(any((self.xor[code == 1] & 0xff803fc0) != 0) * packet_header_0,
                           any((self.xor[code == 2] & 0xfff00700) != 0) * packet_header_1)
        
        # Find which subpackets, if any, have some error, or even just unknowns
        subpacket_errors = array([tri_logic[(subpack == i) & subp_any].max(initial=0) for i in range(12)])
        
        # Find the first subpacket, if any, that definitely has an error
        if any(subpacket_errors == 2):
            first_subpacket = (subpacket_errors == 2).nonzero()[0][0]
        else:
            first_subpacket = -1
        
        # Examine the headers and data of that first subpacket with an error
        head0 = (subpack == first_subpacket) & ((subcode == 5) | (subcode == 7))
        head1 = (subpack == first_subpacket) & (subcode == 6)
        first_subp_head_0 = tri_logic[head0].max(initial=0)
        first_subp_head_1 = tri_logic[head1].max(initial=0)
        
        first_subp_CRC   = any((self.xor[head0] & 0x20000000) != 0) * first_subp_head_0
        first_subp_CM    = any((self.xor[head0] & 0x01ffffe0) != 0) * first_subp_head_0
        first_subp_head  = any((self.xor[head0] & 0xde000000) != 0) * first_subp_head_0
        first_subp_chmap = max(any((self.xor[head0] & 0x0000001f) != 0) * first_subp_head_0, first_subp_head_1)
        first_subp_data = tri_logic[(subpack == first_subpacket) & (subcode == 8)].max(initial=0)
            
        # Check whether any later subpackets had errors, too
        later_subp_errors = subpacket_errors[first_subpacket+1:].max(initial=0)
        
        # Packet CRC and Mandatory IDLE
        CRC  = tri_logic[code == 3].max(initial=0)
        Mand = tri_logic[code == 4].max(initial=0)

        #                       13           12               11               10                  9               8
        summary = array([first_subp_CRC, first_subp_CM, first_subp_head, first_subp_chmap, first_subp_data, later_subp_errors,
                         Payload_Length, Header_Hamming, L1A_orbit, Header_CRC, Event_Header, CRC, Mand, idle])
        #                        7            6              5           4          3          2     1     0
        
        self.pattern = int(sum(summary * 10**arange(summary.shape[0])[::-1]))
        
    def __str__(self):
        outputs = []
        with hexprint():
            for row in range(self.N_row):
                output_row = [self.prebeam_packet[row], ' ' if self.known[row].all() else 'G']
                if self.cap[row].any():
                    output_row.append(self.ASIC_packet[row])
                    
                if any(self.mismatch[row]):
                    output_row.append(str(self.xor[row]).translate(str.maketrans({'0': '-'})))
                outputs.append(' '.join([str(x) for x in output_row]))
        return '\n'.join(outputs)

In [ ]:
# This only needs to be run once, to concatenate the original raw data
# files, and to do a bit of processing to extract the fields packed
# into the second "counter" link_capture channel.

original_data_files_location = '/home/jsw/src/econd-sw'
SEU_MC_file = '/home/jsw/src/econd-sw/SEU_MC_uint32.npz'

if not pathlib.Path(SEU_MC_file).is_file():
    ASIC = zeros((0, 6), dtype=uint32)
    EMU = zeros((0, 6), dtype=uint32)
    counter = zeros((0, 2), dtype=uint32)
    for i in trange(12):
        with np.load(pathlib.Path(original_data_files_location, f"data_{i:d}.npz")) as f:
            ASIC    = concatenate([ASIC,    asarray(f['asic'],    uint32)])
            EMU     = concatenate([EMU,     asarray(f['emu'],     uint32)])
            counter = concatenate([counter, asarray(f['counter'], uint32)])

    cutoff = (diff((counter[:,1] >> 22) & 0x3) == uint32(-3)).nonzero()[0][0] + 1

    counter_raw = counter
    counter = counter_raw[:,0]
    error_bit  = asarray((counter_raw[:,1] >>  0) & ((1 <<  6) - 1), uint8)
    error_word = asarray((counter_raw[:,1] >>  6) & ((1 <<  4) - 1), uint8)
    error_BX   = asarray((counter_raw[:,1] >> 10) & ((1 << 12) - 1), uint16)
    species    = asarray((counter_raw[:,1] >> 22) & ((1 <<  2) - 1), uint8)

    splits = asarray(diff(counter_raw[:cutoff,1]).nonzero()[0]+1, uint32)

    np.savez_compressed(SEU_MC_file,
                        ASIC        = ASIC[:cutoff],
                        EMU         = EMU[:cutoff],
                        counter_raw = counter_raw[:cutoff],
                        counter     = counter[:cutoff],
                        error_bit   = error_bit[:cutoff],
                        error_word  = error_word[:cutoff],
                        error_BX    = error_BX[:cutoff],
                        species     = species[:cutoff],
                        splits      = splits)
    
    del ASIC, EMU, counter, cutoff, counter_raw, error_bit, error_word, error_BX, species, splits

In [ ]:
%%time
hexa_files_location = '/home/jsw/tmp/ECOND_Jan2024_SEU_logs/logs/'
with open(pathlib.Path(hexa_files_location, hexa42_files[0])) as jsonfile:
    J = json.load(jsonfile)
    prebeam = packet_parser(J)

In [ ]:
def make_capture_batches(k, ASIC, EMU, split_inds, percap, prebeam, swap=True):
    start, end = split_inds_padded[[k, k+1]]
    
    if swap:
        our_ASIC = EMU [start:end]
        our_EMU  = ASIC[start:end]
    else:
        our_ASIC = ASIC[start:end]
        our_EMU  = EMU [start:end]
        
    mismatch = any(our_ASIC ^ our_EMU, axis=1)
    last_trigger = -inf
    batches = []
    for i in range(end-start):
        if mismatch[i] and (i > last_trigger + percap):
            # print(start+i-3, start+i-3+percap)
            batches.append(capture_batch(our_ASIC[i-3 : i-3+percap],
                                         our_EMU [i-3 : i-3+percap],
                                         percap,
                                         prebeam))
            last_trigger = i
            
    return items(batches)

def make_patt(k, ASIC, EMU, split_inds, percap, prebeam, swap=True):
    batches = make_capture_batches(k, ASIC, EMU, split_inds, percap, prebeam, swap=True)
    inc = incident(batches, percap, prebeam)
    return inc.pattern

In [ ]:
def find_gaps(mismatch, percap):
    gap_stencil = arange(percap+1) > 0
    old_gap = ones_like(mismatch)
    new_gap = zeros_like(mismatch)
    iteration = 0
    while not all(old_gap == new_gap):
        old_gap = new_gap
        new_gap = convolve(mismatch & ~old_gap, gap_stencil)[:-percap]
    return new_gap

In [ ]:
%%time
with np.load(SEU_MC_file) as f:
    ASIC = f['ASIC']
    EMU = f['EMU']
    counter = f['counter']
    counter_raw = f['counter_raw']
    error_bit = f['error_bit']
    error_word = f['error_word']
    error_BX = f['error_BX']
    species = f['species']
    split_inds = f['splits']
    split_inds_padded = append(insert(split_inds, 0, 0), ASIC.shape[0])

In [ ]:
incident_lengths = awkward.run_lengths(counter_raw[:,1])

In [ ]:
e_bits = awkward.unflatten(error_bit, incident_lengths)[:,0].to_numpy()
e_words = awkward.unflatten(error_word, incident_lengths)[:,0].to_numpy()
e_BXs = awkward.unflatten(error_BX, incident_lengths)[:,0].to_numpy()
e_specs = awkward.unflatten(species, incident_lengths)[:,0].to_numpy()
ak_ASIC = awkward.unflatten(ASIC, incident_lengths)
ak_EMU  = awkward.unflatten(EMU, incident_lengths)

# Find the captures

For each simulated SEU, decide which of the rows that we actually captured would have been captured on Jan. 27, taking capture lengths (5 or 8 BX) and capture gaps / deadtime into account.

In [ ]:
%%time
mismatch = any(ASIC != EMU, axis=1)

gap5 = find_gaps(mismatch, 5)
gap8 = find_gaps(mismatch, 8)

cap5 = append(gap5[4:], zeros(4, bool))
cap8 = append(gap8[4:], zeros(4, bool))

incident_lengths5 = awkward.run_lengths(counter_raw[:,1][cap5])
incident_lengths8 = awkward.run_lengths(counter_raw[:,1][cap8])

ASIC8 = awkward.unflatten(ASIC[cap8], incident_lengths8)
EMU8  = awkward.unflatten(EMU [cap8], incident_lengths8)

ASIC5 = awkward.unflatten(ASIC[cap5], incident_lengths5)
EMU5  = awkward.unflatten(EMU [cap5], incident_lengths5)

# Find the prebeam indices

Map the 32-bit counter onto the prebream data by unrolling its rollovers.

In [ ]:
counter_jump = cumsum(insert((diff(asarray(counter, int64)) < 0), 0, 0))
counter_jump[6309470:] += 5 # This was inserted by hand to make it work out right -- here we apparently went several minutes with no recorded SEU
unrolled_counter = (counter_jump * 2**32 + counter)
prebeam_indices = (unrolled_counter - 67) % 3564 + 130

In [ ]:
packet = awkward.min(awkward.unflatten(prebeam.packet_num.min(axis=1)[prebeam_indices], incident_lengths), axis=1).to_numpy()

In [ ]:
cap_rows5 = awkward.unflatten(prebeam_indices[cap5], incident_lengths5)
first_row5 = cap_rows5[:,0].to_numpy()
last_row5  = cap_rows5[:,-1].to_numpy()

In [ ]:
cap_rows8 = awkward.unflatten(prebeam_indices[cap8], incident_lengths8)
first_row8 = cap_rows8[:,0].to_numpy()
last_row8  = cap_rows8[:,-1].to_numpy()

In [ ]:
packet5 = awkward.min(awkward.unflatten(prebeam.packet_num.min(axis=1)[prebeam_indices[cap5]], incident_lengths5), axis=1).to_numpy(allow_missing=False)
packet8 = awkward.min(awkward.unflatten(prebeam.packet_num.min(axis=1)[prebeam_indices[cap8]], incident_lengths8), axis=1).to_numpy(allow_missing=False)

# False pileup

Find the total number of possible false pileup events when capturing 8 or 5 BX at a time

In [ ]:
N_double8 = zeros(67, uint64)
for k in trange(67):
    N_double8[k] = sum((first_row8[packet8 == k][:,newaxis] - last_row8[packet8 == k][newaxis,:]) > 0)
sum(N_double8)

In [ ]:
N_double5 = zeros(67, uint64)
for k in trange(67):
    N_double5[k] = sum((first_row5[packet5 == k][:,newaxis] - last_row5[packet5 == k][newaxis,:]) > 0)
sum(N_double5)

Randomly sample from all possible false pileup events, and calculate the incident category for each one that we sample.

In [ ]:
%%time
double_patt8 = []
double_i8 = []
double_j8 = []
N = 100000
prog = tqdm(total=N)
while len(double_i8) < N:
    i, j = randint(len(packet), size=2)
    if i != j and packet8[i] == packet8[j] and packet8[i] != 255 and packet8[j] != 255:
        if first_row8[j] > last_row8[i]:
            inc = incident2(concatenate([ASIC8[i].to_numpy(),
                                         ASIC8[j].to_numpy()]),
                    concatenate([EMU8[i].to_numpy(),
                                 EMU8[j].to_numpy()]),
                    concatenate([cap_rows8[i].to_numpy(),
                                 cap_rows8[j].to_numpy()]),
                    packet[i],
                    prebeam)
            double_patt8.append(inc.pattern)
            double_i8.append(i)
            double_j8.append(j)
            prog.update()

In [ ]:
%%time
double_patt5 = []
double_i5 = []
double_j5 = []
N = 100000
prog = tqdm(total=N)
while len(double_i5) < N:
    i, j = randint(len(packet), size=2)
    if i != j and packet5[i] == packet5[j] and packet5[i] != 255:
        if first_row5[j] > last_row5[i]:
            inc = incident2(concatenate([ASIC5[i].to_numpy(),
                                         ASIC5[j].to_numpy()]),
                    concatenate([EMU5[i].to_numpy(),
                                 EMU5[j].to_numpy()]),
                    concatenate([cap_rows5[i].to_numpy(),
                                 cap_rows5[j].to_numpy()]),
                    packet[i],
                    prebeam)
            double_patt5.append(inc.pattern)
            double_i5.append(i)
            double_j5.append(j)
            prog.update()

Save the false pileup events to an output file.

In [ ]:
savez_compressed('false_pileup.npz',
                 double_patt8=array(double_patt8, uint64), double_i8=array(double_i8, uint32), double_j8=array(double_j8, uint32), total8=array([N_double8], uint64),
                 double_patt5=array(double_patt5, uint64), double_i5=array(double_i5, uint32), double_j5=array(double_j5, uint32), total5=array([N_double5], uint64))

### Notes about how to normalize false pileup events

$$λ_\text{orbit} = σ φ t_\text{dwell}$$

$$φ = \frac{\text{fluence}}{t_\text{total}}$$

$$λ = λ_\text{orbit} \frac{t_\text{total}}{t_\text{orbit}} = σ \frac{t_\text{dwell}}{t_\text{orbit}} \text{fluence}$$

$$P(n=0) = e^{-λ}$$

$$P(n=1) = λ e^{-λ}$$

$$P(\text{fake pileup}) = P(\text{1 event A}) \left( \sum_{n=0}^\infty P(\text{no events}) \right) P(\text{1 event B})$$

$$P(\text{fake pileup}) = λ_A e^{-λ_A} λ_B e^{-λ_B} \sum_{n=0}^\infty \left(e^{-λ_\text{anything}}\right)^n = λ_A λ_B e^{-λ_A} e^{-λ_B} \frac{1}{1 - e^{-λ_\text{anything}}}$$

$$e^{-ε} \simeq 1 - ε$$

$$λ_\text{fake pileup, orbit} \simeq λ_A (1 - λ_A) λ_B (1 - λ_B) \frac{1}{1 - (1 - λ_\text{anything})} \simeq λ_{A,\text{orbit}} \frac{λ_{B,\text{orbit}}}{λ_\text{anything,orbit}}$$

$$λ_\text{fake pileup, orbit} = \left(φ σ_i t_{i,\text{dwell, A}}\right) \left(φ σ_i t_{i,\text{dwell, B}}\right) \frac{1}{φ σ_i t_{i,\text{dwell, anything}}}$$

$$λ_\text{fake pileup, orbit} = φ \left(σ_i t_{i,\text{dwell, A}}\right) \left(σ_i t_{i,\text{dwell, B}}\right) \frac{1}{σ_i t_{i,\text{dwell, anything}}}$$

$$λ_\text{fake pileup, orbit} = \frac{\text{fluence}}{t_\text{total}} \left(σ_i t_{i,\text{dwell, A}}\right) \left(σ_i t_{i,\text{dwell, B}}\right) \frac{1}{σ_i t_{i,\text{dwell, anything}}}$$

$$λ_\text{fake pileup} = \frac{\text{fluence}}{t_\text{orbit}} \left(σ_i t_{i,\text{dwell, A}}\right) \left(σ_i t_{i,\text{dwell, B}}\right) \frac{1}{σ_i t_{i,\text{dwell, anything}}}$$

$$λ_\text{fake pileup} = \text{fluence} \left(σ_i \frac{t_{i,\text{dwell, A}}}{t_\text{orbit}}\right) \left(σ_i \frac{t_{i,\text{dwell, B}}}{t_\text{orbit}}\right) \frac{1}{σ_i \frac{t_{i,\text{dwell, anything}}}{t_\text{orbit}}}$$

# Double-bit SEUs

Currently only have this calculated for Output Buffer double-bit SEUs.  But we know they can affect the serializer, too.  So if we decide to use these in the analysis, we will need to calculate the serializer double-bit SEUs, too.  May need to give some thought to double-bit SEUs in the PP SRAM and the eRX, also.

## Two-BX offset in Output Buffer

In [ ]:
OB_xor = (ASIC8[e_specs == 2] ^ EMU8[e_specs == 2])
offset_by_two_code = prebeam.code_data[cap_rows8[e_specs == 2] + 2][(OB_xor != 0)]
newASIC8 = awkward.unflatten(awkward.flatten(ASIC8[e_specs==2]) ^ roll(awkward.flatten(OB_xor * (offset_by_two_code!=0)).to_numpy(), 2, axis=0), incident_lengths8[e_specs==2])

In [ ]:
OB_EMU8 = EMU8[e_specs == 2]
OB_cap_rows8 = cap_rows8[e_specs==2]
OB_packet = packet[e_specs==2]
OBdouble_patt8 = [incident2(newASIC8[i].to_numpy(),
          OB_EMU8[i].to_numpy(),
          OB_cap_rows8[i].to_numpy(),
          OB_packet[i],
          prebeam).pattern for i in trange(sum(e_specs==2))]

## Two-bit offset in Output Buffer (capture 8 BX)

In [ ]:
OB_xor8 = (ASIC8[e_specs == 2] ^ EMU8[e_specs == 2])
OB_bit_offset_ASIC8 = awkward.unflatten(awkward.flatten(ASIC8[e_specs==2]) ^ awkward.flatten(OB_xor8 << 1).to_numpy(), incident_lengths8[e_specs==2])

In [ ]:
OB_EMU8 = EMU8[e_specs == 2]
OB_cap_rows8 = cap_rows8[e_specs==2]
OB_packet = packet[e_specs==2]
OB_bit_offset_patt8 = [incident2(OB_bit_offset_ASIC8[i].to_numpy(),
          OB_EMU8[i].to_numpy(),
          OB_cap_rows8[i].to_numpy(),
          OB_packet[i],
          prebeam).pattern for i in trange(sum(e_specs==2))]

In [ ]:
OB_bit_offset_valid8 = awkward.any(awkward.all(OB_xor8 != 2**31, axis=1), axis=1)

## Two-bit offset in Output Buffer (capture 5 BX)

In [ ]:
OB_xor5 = (ASIC5[e_specs == 2] ^ EMU5[e_specs == 2])
OB_bit_offset_ASIC5 = awkward.unflatten(awkward.flatten(ASIC5[e_specs==2]) ^ awkward.flatten(OB_xor5 << 1).to_numpy(), incident_lengths5[e_specs==2])

In [ ]:
OB_EMU5 = EMU5[e_specs == 2]
OB_cap_rows5 = cap_rows5[e_specs==2]
OB_packet = packet[e_specs==2]
OB_bit_offset_patt5 = [incident2(OB_bit_offset_ASIC5[i].to_numpy(),
          OB_EMU5[i].to_numpy(),
          OB_cap_rows5[i].to_numpy(),
          OB_packet[i],
          prebeam).pattern for i in trange(sum(e_specs==2))]

In [ ]:
OB_bit_offset_valid5 = awkward.any(awkward.all(OB_xor5 != 2**31, axis=1), axis=1)

Save the double-bit SEU events to a file.

Currently, these are not used in the analysis, but they could be added in the future if needed.

In [ ]:
savez_compressed('OB_double_SEU.npz',
                 OB_BX_offset_patt8=OBdouble_patt8,
                 OB_bit_offset_patt8=OB_bit_offset_patt8, OB_bit_offset_valid8=OB_bit_offset_valid8,
                 OB_bit_offset_patt5=OB_bit_offset_patt5, OB_bit_offset_valid5=OB_bit_offset_valid5)

# BX frequency by SEU type

In [ ]:
c,e,a = hist(e_BXs[e_specs == 0], bins=arange(3565)-0.5)
xlim(-0.5, 3563.5)
xlabel('BX')
ylabel('eRX errors recorded per BX')
tight_layout()
savefig('eRX_per_BX.png')

In [ ]:
c,e,a = hist(e_BXs[e_specs == 1], bins=arange(3565)-0.5)
xlim(-0.5, 3563.5)
xlabel('BX')
ylabel('PP errors recorded per BX')
tight_layout()
savefig('PP_per_BX.png')

In [ ]:
c,e,a = hist(e_BXs[e_specs == 2], bins=arange(3565)-0.5)
xlim(-0.5, 3563.5)
xlabel('BX')
ylabel('OB errors recorded per BX')
tight_layout()
savefig('OB_per_BX.png')

In [ ]:
c,e,a = hist(e_BXs[e_specs == 3], bins=arange(3565)-0.5)
xlim(-0.5, 3563.5)
xlabel('BX')
ylabel('eTX errors recorded per BX')
tight_layout()
savefig('eTX_per_BX.png')

# Regular SEUs

Here we calculate the incident category for every simulated SEU.  It takes a while.

In [ ]:
all5 = zeros(len(e_bits), dtype=uint64)
all8 = zeros(len(e_bits), dtype=uint64)

In [ ]:
for k in tqdm(arange(len(e_bits))[all5 == 0]):
    all5[k] = incident2(ASIC5[k].to_numpy(),
                        EMU5[k].to_numpy(),
                        cap_rows5[k].to_numpy(),
                        packet[k],
                        prebeam).pattern

In [ ]:
for k in tqdm(arange(len(e_bits))[all8 == 0]):
    all8[k] = incident2(ASIC8[k].to_numpy(),
                        EMU8[k].to_numpy(),
                        cap_rows8[k].to_numpy(),
                        packet[k],
                        prebeam).pattern

In [ ]:
np.savez_compressed('all58.npz', all5=all5, all8=all8, e_bits=e_bits, e_words=e_words, e_BXs=e_BXs, e_specs=e_specs)

In [ ]:
unique(all5).shape[0], unique(all8).shape[0]

In [ ]:
patt, c = unique(all5, return_counts=True)
c

In [ ]:
patt, c = unique(all8, return_counts=True)
c